In [14]:
import numpy as np 
import pandas as pd 

In [15]:
data = pd.read_csv('/kaggle/input/imdb-dataset-of-top-1000-movies-and-tv-shows/imdb_top_1000.csv', index_col=False)

In [16]:
data.head(5)

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"


In [21]:
movies_desc = data[['Series_Title', 'Overview']]
movies_desc = movies_desc.rename(columns={'Series_Title':'Movie Name', 'Overview':'Description'})
movie_dict = dict(zip(movies_desc["Movie Name"], movies_desc["Description"]))

**Tokenisation and Embeddings**

In [25]:
from sentence_transformers import SentenceTransformer

# Load embedding model
model = SentenceTransformer("intfloat/e5-base")


2025-06-10 09:18:43.607979: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749547123.923005      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749547124.024035      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/356 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [28]:
descriptions = list(movie_dict.values())


In [29]:
embeddings = model.encode(descriptions, convert_to_tensor=True)


Batches:   0%|          | 0/32 [00:00<?, ?it/s]

In [30]:
embeddings

tensor([[-0.0369,  0.0219, -0.0012,  ...,  0.0175,  0.0133,  0.0013],
        [-0.0754,  0.0234, -0.0175,  ..., -0.0116,  0.0264,  0.0416],
        [-0.0683,  0.0316, -0.0175,  ..., -0.0011,  0.0066,  0.0177],
        ...,
        [-0.0475,  0.0201, -0.0299,  ...,  0.0193,  0.0188,  0.0359],
        [-0.0443, -0.0247, -0.0039,  ...,  0.0106,  0.0296,  0.0374],
        [-0.0614, -0.0033, -0.0050,  ...,  0.0262,  0.0003,  0.0419]])

In [32]:
import torch

movie_data = list(zip(movies_desc["Movie Name"], embeddings))


In [38]:
len(movie_data)

999

In [39]:
torch.save(embeddings, "movie_embeddings.pt")
movies_desc.to_csv("/kaggle/working/movies.csv", index=False)

In [51]:
user_query = "I want a romantic movie with a twist ending."
user_embedding = model.encode(user_query, convert_to_tensor=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [52]:
from torch.nn.functional import cosine_similarity

similarities = cosine_similarity(user_embedding, embeddings)

In [45]:
similarities

tensor([0.6975, 0.7042, 0.7145, 0.6905, 0.6999, 0.6915, 0.7417, 0.6838, 0.7176,
        0.7270, 0.7088, 0.7238, 0.7299, 0.6814, 0.7461, 0.7076, 0.6956, 0.7152,
        0.6827, 0.6924, 0.7229, 0.6965, 0.6906, 0.7111, 0.6982, 0.7105, 0.7233,
        0.7303, 0.7534, 0.6976, 0.7008, 0.6733, 0.6921, 0.7079, 0.7016, 0.7069,
        0.7224, 0.6953, 0.6733, 0.7190, 0.6860, 0.7410, 0.7033, 0.6895, 0.7130,
        0.7256, 0.6742, 0.6939, 0.7338, 0.7140, 0.7123, 0.6675, 0.7089, 0.6799,
        0.7273, 0.7143, 0.7314, 0.6801, 0.7164, 0.7209, 0.7188, 0.6996, 0.7263,
        0.7150, 0.7053, 0.6818, 0.7101, 0.7176, 0.7149, 0.7189, 0.7114, 0.7178,
        0.6931, 0.7474, 0.7112, 0.7286, 0.7568, 0.7107, 0.7220, 0.7352, 0.6573,
        0.7152, 0.7812, 0.6867, 0.7200, 0.7231, 0.7222, 0.7356, 0.7310, 0.6932,
        0.7178, 0.7401, 0.6769, 0.6995, 0.7182, 0.7005, 0.6938, 0.7020, 0.7138,
        0.7141, 0.7137, 0.6927, 0.6746, 0.7307, 0.6848, 0.7115, 0.7181, 0.6716,
        0.7006, 0.7062, 0.7145, 0.7166, 

In [44]:
movies_desc.head(5)

,Movie Name,Description
0,The Shawshank Redemption,Two imprisoned men bond over a number of years...
1,The Godfather,An organized crime dynasty's aging patriarch t...
2,The Dark Knight,When the menace known as the Joker wreaks havo...
3,The Godfather: Part II,The early life and career of Vito Corleone in ...
4,12 Angry Men,A jury holdout attempts to prevent a miscarria...


In [53]:
scores = cosine_similarity(user_embedding, embeddings)

for idx in top_indices:
    i = idx.item()  # convert from tensor to int
    print(f"{movies_desc.iloc[i]['Movie Name']}\n→ {movies_desc.iloc[i]['Description']}\n")

Sunset Blvd.
→ A screenwriter develops a dangerous relationship with a faded film star determined to make a triumphant return.

The Big Short
→ In 2006-2007 a group of investors bet against the US mortgage market. In their research they discover how flawed and corrupt the market is.

In the Heat of the Night
→ A black police detective is asked to investigate a murder in a racially hostile southern town.

Per un pugno di dollari
→ A wandering gunfighter plays two rival families against each other in a town torn apart by greed, pride, and revenge.

Almost Famous
→ A high-school boy is given the chance to write a story for Rolling Stone Magazine about an up-and-coming rock band as he accompanies them on their concert tour.

